인공지능 작사가 만들기
1.데이터 불러오기

In [1]:
import re
import numpy as np
import tensorflow as tf
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)


raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 리스트에 문장 단위로 저장 
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()   # 텍스트를 라인 단위로 끊어서 읽어오기
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:30])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?", 'It goes like this', 'The fourth, the fifth', 'The minor fall, the major lift', 'The baffled king composing Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah', 'Hallelujah Your faith was strong but you needed proof', 'You saw her bathing on the roof', 'Her beauty and the moonlight overthrew her', 'She tied you', 'To a kitchen chair', 'She broke your throne, and she cut your hair', 'And from your lips she drew the Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah', 'Hallelujah You say I took the name in vain', "I don't even know the name", "But if I did, well really, what's it to you?", "There's a blaze of light", 'In every word', "It doesn't matter which you heard", 'The holy or the broken Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah', "Hallelujah I did my best, it wasn't much", "I couldn't feel, so I tried to touch", "

2.데이터 정제하기

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    
    raw_corpus.append(sentence)
    
    if idx > 30: break   # 문장 30개만 확인
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof
You saw her bathing on the roof
Her beauty and the moonlight overthrew her
She tied you
To a kitchen chair
She broke your throne, and she cut your hair
And from your lips she drew the Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah You say I took the name in vain
I don't even know the name
But if I did, well really, what's it to you?
There's a blaze of light
In every word
It doesn't matter which you heard
The holy or the broken Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah I did my best, it wasn't much
I couldn't feel, so I tried to touch
I've told the truth, I didn't come to fool you
And even though


3. 토큰화 하기

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)       # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)              # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'      # 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여줌
    return sentence

In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
     
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
      

corpus[:30]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>',
 '<start> you saw her bathing on the roof <end>',
 '<start> her beauty and the moonlight overthrew her <end>',
 '<start> she tied you <end>',
 '<start> to a kitchen chair <end>',
 '<start> she broke your throne , and she cut your hair <end>',
 '<start> and from your lips she drew the hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah you say i took the name in vain <end>',
 '<start> i don t even know the name <end>',
 '<start> bu

In [5]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=15000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있음. 이번에는 사용하지 안함
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 <unk> 처리
    )
    tokenizer.fit_on_texts(corpus)   # 구축한 corpus로부터 Tokenizer가 사전을 자동구축

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환.
   
    for num in tensor:
        if len(num) >= 29:
            tensor = np.delete(tensor, num)
            
    print(len(tensor))
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=15)  

    print(tensor,tokenizer)
    return tensor, tokenizer

    
tensor, tokenizer = tokenize(corpus)

/opt/conda/lib/python3.9/site-packages/numpy/lib/function_base.py:4454: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)


152430
[[  2  50   5 ...   0   0   0]
 [  2   5  22 ...   0   0   0]
 [  2  39  39 ...   0   0   0]
 ...
 [  2   5  91 ... 561   7   3]
 [  2   8 163 ...   0   0   0]
 [  2  11  24 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f74e72e70a0>


In [6]:
# 단어 사전 어떻게 생겼는지 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break
        

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [7]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성.

print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  297   64   57    9  962 5633    3    0    0    0]
[  50    5   91  297   64   57    9  962 5633    3    0    0    0    0]


4. 평가 데이터셋 분리하기

In [8]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          train_size = 0.2,
                                                          random_state=34)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (30486, 14)
Target Train: (30486, 14)


In [9]:
BUFFER_SIZE = len(src_input)         
BATCH_SIZE = 256                     
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 15000개와, 여기 포함되지 않은 0:<pad>를 포함하여 15001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

5. 모델 학습하기

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.drop  = tf.keras.layers.Dropout(0.5)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.drop(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 15001), dtype=float32, numpy=
array([[[ 1.02680482e-04,  1.53312096e-04,  1.44893362e-04, ...,
          6.47222041e-05,  1.22277139e-04, -4.43175159e-05],
        [ 3.12377815e-04,  3.41877196e-04,  1.85570563e-04, ...,
          2.24266609e-04,  2.16036438e-04, -1.80199262e-04],
        [ 5.49899705e-04,  3.75131582e-04,  2.34697989e-04, ...,
          3.51029332e-04,  2.02147115e-04, -3.26054462e-04],
        ...,
        [ 6.85353356e-04,  1.26634550e-03, -1.07466294e-04, ...,
          3.29018134e-04,  4.69532446e-04, -3.57399607e-04],
        [ 7.66806595e-04,  1.08826894e-03, -4.14961280e-04, ...,
          2.00115843e-04,  4.98281443e-05, -2.88796058e-04],
        [ 1.06092752e-03,  1.15421298e-03, -5.64936025e-04, ...,
          1.32507223e-04, -2.05121803e-04, -2.83112284e-04]],

       [[-1.80896459e-04,  2.47294192e-05,  2.03897493e-04, ...,
         -1.04676037e-04,  8.08490295e-05,  7.59679679e-05],
        [ 7.16725253e-06,  3.49015208e-05,  1

In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3840256   
_________________________________________________________________
lstm (LSTM)                  multiple                  18882560  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  30737049  
Total params: 87,022,489
Trainable params: 87,022,489
Non-trainable params: 0
_________________________________________________________________


In [13]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
595/595 [==============================] - 293s 489ms/step - loss: 3.6525
Epoch 2/10
595/595 [==============================] - 296s 497ms/step - loss: 3.1075
Epoch 3/10
595/595 [==============================] - 296s 497ms/step - loss: 2.8651
Epoch 4/10
595/595 [==============================] - 296s 498ms/step - loss: 2.6544
Epoch 5/10
595/595 [==============================] - 297s 499ms/step - loss: 2.4582
Epoch 6/10
595/595 [==============================] - 297s 498ms/step - loss: 2.2765
Epoch 7/10
595/595 [==============================] - 297s 499ms/step - loss: 2.1093
Epoch 8/10
595/595 [==============================] - 297s 499ms/step - loss: 1.9589
Epoch 9/10
595/595 [==============================] - 297s 499ms/step - loss: 1.8229
Epoch 10/10
595/595 [==============================] - 297s 499ms/step - loss: 1.7015


6. 모델 평가하기

In [14]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [15]:
generate_text(model, tokenizer, init_sentence="<start> me")

'<start> me and my homies , so drop that <end> '

In [16]:
generate_text(model, tokenizer, init_sentence="<start> yes")

'<start> yes , i am , yes i am . <end> '

In [17]:
generate_text(model, tokenizer, init_sentence="<start> my")

'<start> my name is <end> '

In [18]:
generate_text(model, tokenizer, init_sentence="<start> you")

'<start> you re the only one i want <end> '

In [19]:
generate_text(model, tokenizer, init_sentence="<start> hey")

'<start> hey , hey , hey <end> '

회고
커널 연결이 계속 끊어져서 중단되어 다시 만드는 일이 반복되어 많은 시간이 소모되었다.
그리고 epochs값을 30에서 10으로 하여 학습 시간을 줄였다. validation loss가  1.7이 나왔다.새롭고 신기한 문장을 만드니 즐거운 프로젝트였고 뜻깊은 시간이었습니다.